# Testing out OPT on Self-Ask Decomposition

Lets see how large of a model we can effectively use!

In [1]:
from transformers import AutoTokenizer
from llm_serving.model.wrapper import get_model

In [ ]:
#size="13b"
# Load the tokenizer. All OPT models with different sizes share the same tokenizer
tokenizer = AutoTokenizer.from_pretrained(f"facebook/opt-66b")
tokenizer.add_bos_token = False

# Load the model. Alpa automatically downloads the weights to the specificed path

model = get_model(model_name=f"alpa/opt-66b", path="/scratch/ddr8143/opt_weights/")

Downloading:   0%|          | 0.00/685 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/676 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.36G [00:00<?, ?B/s]

In [ ]:
# Generate
prompt = """Question: Harmony Korine was both screenwriter and director of what movie?
Can this be decomposed: Yes.
Question 1: Harmony Korine was the screenwriter of what movie?
Question 2: Harmony Korine was the director of what movie?
So the final answers are: Spring Breakers, Trash Humpers, Julien Donkey-Boy, Mister Lonely, The Beach Bum, Gummo

Question: Who was both a graduate from Ananda College and University of Ceylon?
Can this be decomposed: Yes.
Question 1: Who graduated from Ananda College?
Question 2: Who graduated from University of Ceylon?
So the final answers are: Tissa Vitharana, V. K. Samaranayake, Chandre Dharma-wardana, D. Shelton A. Gunaratne, K. D. Arulpragasam, J. B. Disanayake

Question: Which movie had K. S. L. Swamy as its director and Vijaya Bhaskar as its musical composer?
Can this be decomposed: Yes.
Question 1: Which movie has K. S. L. Swamy as its director?
Question 2: Which movie has Vijaya Bhaskar as its musical composer?
So the final answers are: Anna Thamma, Thulasi, Devaru Kotta Thangi, Manku Dinne, Lagna Pathrike, Malaya Marutha, Mugdha Manava, Banashankari, Aaru Mooru Ombhatthu, Bhagya Jyothi, Bhagyada Bagilu, Bhale Adrushtavo Adrushta, Maagiya Kanasu, Mithileya Seetheyaru, Arishina Kumkuma, Makkala Bhagya, Jimmy Gallu, Huli Hejje, CID 72, Lakshmi Saraswathi, Thoogudeepa

Question: What music was composed by Devi Sri Prasad and  produced by Dil Raju?
Can this be decomposed: Yes.
Question 1: What music was composed by Devi Sri Prasad?
Question 2: What music was produced by Dil Raju?
So the final answers are: Arya, Bommarillu, Nenu Local, Mr. Perfect, Bhadra, Yevadu, Maharshi, Middle Class Abbai

Question: What movie did Irwin Allen both direct and produce?
Can this be decomposed:"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
output = model.generate(
    input_ids=input_ids,
    max_length=600,
    do_sample=True,
)
generated_string = tokenizer.batch_decode(output, skip_special_tokens=True)

print(generated_string[0])

In [1]:
import jsonlines
import json
import numpy as np
import random
import openai
from collections import defaultdict

import multiqa_utils.openai_utils as ou
import multiqa_utils.qampari_utils as qu
import multiqa_utils.decomposition_utils as du

%load_ext autoreload
%autoreload 2

In [3]:
qmp_dev = qu.load_dev_data()
qmp_train = qu.load_train_data()
print("Num Dev Qs:", len(qmp_dev))
print("Num Train Qs:", len(qmp_train))

Num Dev Qs: 1000
Num Train Qs: 61911


In [4]:
qmp_train_wd = [d for d in qmp_train if 'wikitables' not in d['qid']]

In [5]:
len(qmp_train_wd)

56075

In [7]:
## Load my manual decompositions to use for prompt creation
output_dir = "/scratch/ddr8143/multiqa/qampari_data/decomposition_v0/"
manual_train_decomp_file = f"{output_dir}manual_decompositions_train.json"
#json.dump(official_train_decomp, open(manual_train_decomp_file, 'w+'))
manual_train_decomp = {int(k): v for k, v in json.load(open(manual_train_decomp_file)).items()}

In [9]:
curr_best_prompt = du.get_qmp_decomp_prompt_base_v1(qmp_train_wd, manual_train_decomp)

In [10]:
print(curr_best_prompt)

Instructions:
Choose the question type out of: composition or intersection.

Simple questions only require one piece of information. Example:

“Question: Which software, art, etc. has Don Broco as performer?
Question Type: simple.
Explanation: This is a simple question because we only need to know what Don Broco has performed in.”

Composition questions require getting one answer and then getting more information about that answer. Example:

“Question: What are the dates of death of persons that were a member of the political party Australian Labor Party (Anti-Communist)?
Question Type: composition.
Explanation: This is a complosition question because we need to know who were members of the political party and then we need to get additional information about each of them.”

Intersection questions require getting answers to two questions and then combining them. Example:

“Question: Which film has M. G. Ramachandran as a member of its cast and has J. Jayalalithaa as a member of its cast

In [ ]:
mu.decomp_prompt_base(path_args)